### Dependendies

##### pyodbc: Provides an interface to connect Python with ODBC databases,such as Microsoft SQL Server, allowing execution of SQL queries.
##### sqlalchemy: Offers a SQL toolkit and Object-Relational Mapping (ORM) capabilities, simplifying database operations and query execution via Python.
##### urllib: A standard Python library used for working with URLs, such as opening URLs and handling web-related requests and responses.
##### logging: Part of Python's standard library, it allows tracking events during the execution of the program, useful for debugging and recording program flow or errors.





In [1]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
import logging
import pyodbc
import urllib

### Setting up logging
##### In this project, logging is used to track events, errors, and the overall flow of the application. The Python `logging` module is configured to write log messages to a file. This helps with debugging and auditing the execution of the program.

In [2]:

logging.basicConfig(filename='data_processing.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Function to log errors
def log_error(error_message):
    logging.error(f"Error: {error_message}")

### Step 1: Load the CSV

##### The first step in the data processing pipeline is loading the CSV file. This project uses the `pandas` library, which provides an efficient and easy way to load and manipulate CSV data.

In [3]:

def load_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='unicode_escape')
        logging.info("CSV file loaded successfully.")
        return df
    except Exception as e:
        log_error(f"Failed to load CSV: {e}")
        raise e

### Step 2: Establish a connection to SQL Server

##### After loading the CSV file, the next step is to establish a connection to the SQL Server. This project uses the `pyodbc` and `sqlalchemy` libraries to connect to a Microsoft SQL Server database.

In [4]:

def create_sql_connection(server, database):
    try:
        # Using SQLAlchemy to create an engine
        engine = create_engine(f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server")
        conn = pyodbc.connect(
            trusted_connected='yes',
            DRIVER='SQL Server',
            SERVER=server,
            DATABASE=database   
        )
        logging.info("Connected to SQL Server successfully.")
        return conn, engine
    except Exception as e:
        log_error(f"Failed to connect to SQL Server: {e}")
        raise e

### Step 3: Fetch data from the SQL table

##### Once a connection to the SQL Server is established, the next step is to fetch data from the database. This project uses the `pandas` library in combination with the SQLAlchemy engine to retrieve data from a specific SQL table and load it into a DataFrame for further processing.

In [5]:

def fetch_data_from_sql(conn, table_name):
    try:
        query = f"SELECT * FROM {table_name}"
        df_sql = pd.read_sql(query, conn)
        logging.info(f"Data fetched from {table_name} successfully.")
        return df_sql
    except Exception as e:
        log_error(f"Failed to fetch data from SQL: {e}")
        raise e

### Step 4: Data cleaning

##### After fetching the data from the SQL table, it’s important to clean the data to ensure it is suitable for analysis or further processing. Data cleaning involves handling missing values, correcting data types, removing duplicates, and addressing any inconsistencies in the dataset.

In [6]:

def clean_data(df_sql):
    try:
        # Dropping the 'Unnamed' and 'Status' columns
        df_sql.drop(columns=['Unnamed', 'Status'], inplace=True, errors='ignore')

        # Renaming 'nage' to 'age_group'
        df_sql.rename(columns={'nage': 'age_group'}, inplace=True)

        # Dropping duplicates
        df_sql.drop_duplicates(inplace=True)

        # Resetting the index
        df_sql.reset_index(drop=True, inplace=True)

        logging.info("Data cleaning and transformation completed successfully.")
        return df_sql
    except Exception as e:
        log_error(f"Data cleaning failed: {e}")
        raise e

### Step 5: Save the updated data back to the SQL Server

##### After cleaning and processing the data, the final step is to save the updated data back into the SQL Server database. This project uses the `to_sql()` function from the `pandas` library in combination with SQLAlchemy to write the DataFrame back to the SQL table.

In [7]:

def save_data_to_sql(df, engine, table_name):
    try:
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        logging.info(f"Data saved back to {table_name} successfully.")
    except Exception as e:
        log_error(f"Failed to save data to SQL Server: {e}")
        raise e

### Main function to execute the workflow

##### The main function serves as the entry point for executing the entire data processing workflow. It orchestrates the sequence of operations, including loading the CSV, establishing a connection to SQL Server, fetching data, cleaning the data, and saving the updated data back to the SQL Server.

In [8]:
def main():
    csv_file_path = r"C:\Users\LENOVO\OneDrive\Desktop\deeppython\Python_Diwali_Sales_Analysis-main\data processor\Diwali.csv"
    server = 'LAPTOP-U3795DN8'
    database = 'mydatabase'
    table_name = 'dbo.Diwali'

    # Load CSV data
    df = load_csv(csv_file_path)

    # Connect to SQL Server
    conn, engine = create_sql_connection(server, database,)

    # Fetch data from SQL
    df_sql = fetch_data_from_sql(conn, table_name)

    # Clean data
    cleaned_df = clean_data(df_sql)

    # Save the cleaned data back to SQL
    save_data_to_sql(cleaned_df, engine, table_name)

    # Close the connection
    conn.close()
    logging.info("Database connection closed.")

def create_sql_connection(server, database):
    conn_str = (
        f"Driver={{ODBC Driver 17 for SQL Server}};"
        f"Server={server};"
        f"Database={database};"
        "Trusted_Connection=yes;"
    )
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(conn_str)}")
    conn = pyodbc.connect(conn_str)
    return conn, engine


if __name__ == "__main__":
    main()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20240\4278941122.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, conn)
